In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import StratifiedKFold,KFold,GridSearchCV  
import joblib
import numpy as np
import os
from lofo import LOFOImportance, Dataset, plot_importance,FLOFOImportance
from sklearn.metrics import f1_score
import sklearn
import datetime
import catboost as cb
import warnings
warnings.filterwarnings("ignore")
# pd.set_option('display.max_columns', 100) # 设置显示最大列数
pd.set_option('display.max_rows', 300) # 设置显示最大行数

In [ ]:
data = pd.read_csv('../data/data_4feas_plus_gere.csv')

In [ ]:
feas=[i  for i in data.columns.tolist() if i not in ['id','label']]

x_train = data[feas]
y_train = data['label']
folds = 5
seed = 2023#1111
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

cv_scores = []
test_pre = []
Feass = pd.DataFrame()
train = np.zeros(x_train.shape[0])
models = []

current_time = datetime.datetime.now()
current_time = str(current_time).replace(' ','-')
# os.mkdir('../log/'+current_time)
for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
    param_grid = {'metric':'F1','n_estimators': 500, 'learning_rate': 0.2, 'max_depth':7,'num_leaves':127,  'bagging_fraction': 0.7, 'feature_fraction': 0.2}
    # param_grid = {'metric':'F1','n_estimators': 500, 'learning_rate': 0.2, 'max_depth':7,'num_leaves':127,  'bagging_fraction': 0.8, 'feature_fraction': 0.8}
    gbm = lgb.LGBMClassifier(objective="binary", **param_grid)
    gbm.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], eval_metric='l2', callbacks = [lgb.log_evaluation(period=50), lgb.early_stopping(stopping_rounds=50)])
    models.append(gbm)
    # joblib.dump(gbm, f"../log/{current_time}/lgbm_model{i+1}.pkl")
    proba = gbm.predict_proba(val_x, num_iteration=gbm.best_iteration_)
    val_pred = np.where(proba[:,0]>0.83,0,1)
    train[valid_index] = val_pred
    print(f1_score(val_pred,val_y))
f1 = f1_score(train,y_train)
print(f1)
# os.rename(f'../log/{current_time}',f'../log/{f1}{current_time[:-6]}')

In [ ]:
importance = pd.DataFrame(columns=['name','importance'])
importance['name']=models[-1].feature_name_
idx = 0
for gbm in models:
    if(idx==0):
        importance['importance']=gbm.feature_importances_
    else:
        importance['importance']+=gbm.feature_importances_

In [ ]:
imp_dic = {}
for i in range(0,400):
    df = importance[importance['name'].str.startswith(f"{i}_")]
    score = df['importance'].sum()
    imp_dic[i] = score

In [ ]:
sorted_d = sorted(imp_dic.items(), key=lambda x: x[1], reverse=True)
top_50 = sorted_d[:50]
keys = [x[0] for x in top_50]

In [ ]:
coloums_list = []
for i in keys:
    coloums_list.append(f'{i}_min')
    coloums_list.append(f'{i}_max')
    coloums_list.append(f'{i}_mean')
    coloums_list.append(f'{i}_std')
coloums_list.append('label')

In [ ]:
joblib.dump(coloums_list,'../data/coloums_list')

In [ ]:
cut_data = data[coloums_list]

In [ ]:
feas=[i  for i in cut_data.columns.tolist() if i not in ['id','label']]

x_train = cut_data[feas]
y_train = cut_data['label']
folds = 5
seed = 2023
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

# train = np.zeros(train_x.shape[0])

cv_scores = []
test_pre = []
Feass = pd.DataFrame()
train = np.zeros(x_train.shape[0])
models = []

current_time = datetime.datetime.now()
current_time = str(current_time).replace(' ','-')
os.mkdir('../cut_model/'+current_time)
for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
    gbm = lgb.LGBMClassifier(bagging_fraction=0.7, feature_fraction=0.2, learning_rate=0.2,
               max_depth=7, metric='F1', n_estimators=500, num_leaves=127,
               objective='binary')
#     gbm = lgb.LGBMClassifier(bagging_fraction=0.8, feature_fraction=0.8, learning_rate=0.2,
#             max_depth=7, metric='F1', n_estimators=500, num_leaves=127,
#             objective='binary')
    gbm.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], eval_metric='l2', callbacks = [lgb.log_evaluation(period=50), lgb.early_stopping(stopping_rounds=50)])
    models.append(gbm)
    joblib.dump(gbm, f"../cut_model/{current_time}/lgbm_model{i+1}.pkl")
    proba = gbm.predict_proba(val_x, num_iteration=gbm.best_iteration_)
    val_pred = np.where(proba[:,0]>0.85,0,1)
    train[valid_index] = val_pred
    print(f1_score(val_pred,val_y))
f1 = f1_score(train,y_train)
print(f1)
os.rename(f'../cut_model/{current_time}',f'../cut_model/{f1}{current_time[:-6]}')